In [1]:
import glob
import numpy as np
import pandas as pd

In [2]:
files = map(np.load, glob.glob("../export/data/feature_data_*.npz"))

In [3]:
selected_signals = ['ECG', 'EDA',  'Resp', 'SKT']
selected_attribute = ['mean', 'std', 'min', 'max', 'mean_diff', 'mean_abs_diff']

n_selected_signals = len(selected_signals)
n_selected_attribute = len(selected_attribute)

n_subject = len(files)

skip_sample = 100
number_of_sample = 800

In [4]:
x_valence = pd.DataFrame()
x_arousal = pd.DataFrame()
y_valence = []
y_arousal = []

for f in files[:n_subject]:

    session = f["session_info"].tolist()
    sid = int(session["sessionId"])

    mediafile = session["mediaFile"]
    
    valence = int(session["feltVlnc"])
    arousal = int(session["feltArsl"])

    data_valence = pd.DataFrame()
    data_arousal = pd.DataFrame()

    for signal in selected_signals:
        for attribute in selected_attribute:

            # add a column for each feature
            column = signal + "_" + attribute
            data_valence[column] = f["valence"].tolist()[column][skip_sample:skip_sample + number_of_sample]
            data_arousal[column] = f["arousal"].tolist()[column][skip_sample:skip_sample + number_of_sample]

    # concatenate all subjects' data
    x_valence = pd.concat([x_valence, data_valence], ignore_index=True)
    x_arousal = pd.concat([x_arousal, data_arousal], ignore_index=True)

    # add a valence label for each timestep
    y_valence.append(valence)
    y_arousal.append(arousal)

# reshape to CNN input shape
x_valence = x_valence.fillna(x_valence.median())
x_arousal = x_arousal.fillna(x_arousal.median())

x_valence = x_valence.values.reshape(n_subject, number_of_sample, n_selected_attribute*n_selected_signals)
x_arousal = x_arousal.values.reshape(n_subject, number_of_sample, n_selected_attribute*n_selected_signals)

# reduce class numbers
y_valence = pd.DataFrame(y_valence)
y_arousal = pd.DataFrame(y_arousal)

In [6]:
print x_valence.shape
print y_valence.shape

print x_arousal.shape
print y_arousal.shape

(402, 800, 24)
(402, 1)
(402, 800, 24)
(402, 1)


In [7]:
np.save("data_valence_physio_for_cnn.npy", x_valence)
np.save("data_valence_label_for_cnn.npy", y_valence)

np.save("data_arousal_physio_for_cnn.npy", x_arousal)
np.save("data_arousal_label_for_cnn.npy", y_arousal)